In [1]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import QueryEngineTool
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [2]:
llm_35 = OpenAI(model="gpt-3.5-turbo-0613")
llm_4 = OpenAI(model="gpt-4-0613")
# base_agent = ReActAgent.from_tools(query_engine_tools, llm=llm_35, verbose=True)

In [3]:
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)

# gpt-4
step_decompose_transform = StepDecomposeQueryTransform(llm=llm_4, verbose=True)

# gpt-3
step_decompose_transform_gpt3 = StepDecomposeQueryTransform(
    llm=llm_35, verbose=True
)

In [4]:
documents = SimpleDirectoryReader("./query-decomposition/data/filler").load_data()
index = VectorStoreIndex.from_documents(documents, llm=llm_35, verbose=True)

In [21]:
index_summary = "Used to answer questions about the teacher, the students, and the classroom"

In [22]:
# set Logging to DEBUG for more detailed outputs
from llama_index.core.query_engine import MultiStepQueryEngine

query_engine = index.as_query_engine(llm=llm_4)
query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform,
    index_summary=index_summary,
)

In [6]:
import json

file_path = './query-decomposition/cv_loc_enhanced_questions.json'
with open(file_path, 'r') as file_loc:
    data = json.load(file_loc)

new_questions = data

In [12]:
def save_questions(questions, path):
    with open(path, "w") as f:
        for question in questions:
            f.write(question + "\n")
            
def load_questions(path):
    questions = []
    with open(path, "r") as f:
        for line in f:
            questions.append(line.strip())
    return questions

In [10]:
# TODO: split up into cv and loc questions, also train on each possible set of eval/test split
train_questions, eval_questions = new_questions[:60], new_questions[60:]

In [11]:
save_questions(train_questions, "train_questions_10q.txt")
save_questions(eval_questions, "eval_questions_10q.txt")

In [13]:
train_questions = load_questions("train_questions_10q.txt")
eval_questions = load_questions("eval_questions_10q.txt")

In [14]:
from llama_index.llms.openai import OpenAI
from llama_index.finetuning.callbacks import OpenAIFineTuningHandler
from llama_index.core.callbacks import CallbackManager
from llama_index.core.agent import ReActAgent

finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

from llama_index.core import Settings

# limit the context window artifically to test refine process
Settings.context_window = 2048

In [29]:
responses = []
for idx, question in enumerate(train_questions):
    print(f"[{idx}] Question: {question}")
    response = query_engine.query(question)
    responses.append(response)
    print(f"[{idx}] Agent Response: {str(response)}")

# question = train_questions[0]
# response_gpt4 = query_engine.query(
#     "Is the teacher standing in front of the classroom while giving a lecture or conducting an activity?",
# )

[0] Question: Is the teacher standing in front of the classroom while giving a lecture or conducting an activity?
> Current query: Is the teacher standing in front of the classroom while giving a lecture or conducting an activity?
> New query: Is the teacher giving a lecture or conducting an activity?
> Current query: Is the teacher standing in front of the classroom while giving a lecture or conducting an activity?
> New query: Is the teacher standing in front of the classroom?
> Current query: Is the teacher standing in front of the classroom while giving a lecture or conducting an activity?
> New query: None
[0] Agent Response: The context does not provide information about whether the teacher is standing in front of the classroom while giving a lecture or conducting an activity.
[1] Question: How does the classroom arrangement affect the teacher's ability to interact with students at their desks?
> Current query: How does the classroom arrangement affect the teacher's ability to in

In [31]:
for response in responses:   
    sub_qa = response.metadata["sub_qa"]
    tuples = [(t[0], t[1]) for t in sub_qa]
    if len(tuples) > 0:
        latest_tuple = tuples[-1]
        latest_query = latest_tuple[0]
    print(latest_query)

Is the teacher standing in front of the classroom?
What are some common classroom arrangements?
Is the student writing an answer on the whiteboard individually?
What are the titles of the books on the teacher's desk?
How often is the projector used in the classroom?
What are the dynamics of individual student work in the classroom?
Does the context provide information on whether a student raising their hand to ask a question can indicate a high level of classroom participation?
What is the teacher using the map on the wall for?
Does opening windows in the classroom improve air quality?
What is the student using their laptop for?
Is there a holder for pencils and pens in the classroom?
What is the primary source of information the teacher is using?
What is the level of student focus when watching an educational video?
Does the classroom have a clock?
What is the stress level of students when taking a test?
What is the teacher's method for checking homework assignments?
What are the stud

Is the teacher standing in front of the classroom?
